In [1]:
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
import pandas as pd
import tarfile
import numpy as np

In [2]:
##--Dataset
newsgroups = datasets.fetch_20newsgroups(
                        subset='all',
                        categories=['alt.atheism', 'sci.space'])
# newsgroups

In [3]:
##-- Разархивируем файл
# data = tarfile.open('20newsbydate.tar.gz')
# data.extractall()
# data
# newsgroups.data

In [67]:
df1 = pd.DataFrame(newsgroups.data)
df1

,0
0,From: 9051467f@levels.unisa.edu.au (The Desert...
1,From: kempmp@phoenix.oulu.fi (Petri Pihko)\nSu...
2,From: fairfiel@helios.usq.EDU.AU (raymond fair...
3,From: steinly@topaz.ucsc.edu (Steinn Sigurdsso...
4,From: jcobban@bnr.ca (Jim Cobban)\nSubject: Re...
...,...
1781,From: nsmca@aurora.alaska.edu\nSubject: Re: Su...
1782,From: loss@fs7.ECE.CMU.EDU (Doug Loss)\nSubjec...
1783,From: bill@xpresso.UUCP (Bill Vance)\nSubject:...
1784,From: jcm@head-cfa.harvard.edu (Jonathan McDow...


In [16]:
##-- Словам в тексте придаем уникальные числовые значения
vector = TfidfVectorizer()
X = vector.fit_transform(newsgroups.data)
X

<1786x28382 sparse matrix of type '<class 'numpy.float64'>'
	with 303138 stored elements in Compressed Sparse Row format>

In [19]:
## Очень долго считает, чуть комп не взорвался
##-- Находит наилучший 'C'
grid = {'C' : np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv, n_jobs=-1)
gs.fit(X, newsgroups.target)
best_C = gs.best_params_['C']
print(best_C)

1.0


In [21]:
##-- Создаем новый классификатор на лучшем 'C'
##-- Тренируем его
##-- И находим 10 последних индексов, 
##-- в которых самые часто встречающиеся слова
new_clf = SVC(C=1, kernel='linear', random_state=241)
new_clf.fit(X, newsgroups.target)

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=241, shrinking=True, tol=0.001,
    verbose=False)

In [22]:
new_clf.coef_

<1x28382 sparse matrix of type '<class 'numpy.float64'>'
	with 18404 stored elements in Compressed Sparse Row format>

In [26]:
coefs = abs(new_clf.coef_.todense().A1)
coefs

array([0.29258057, 0.12314757, 0.        , ..., 0.01972862, 0.05831336,
       0.00297347])

In [27]:
coefs = np.argsort(coefs)
coefs

array([ 7801, 21437,  9144, ...,  5088, 12871, 24019], dtype=int64)

In [57]:
fin_coefs = coefs[-10:]
vector.get_feature_names()[24019]

'space'

In [61]:
new_list = []
for i in range(10):
    print(vector.get_feature_names()[fin_coefs[i]])
    new_list.append(vector.get_feature_names()[fin_coefs[i]])

sci
keith
bible
religion
sky
moon
atheists
atheism
god
space


In [66]:
sorted(new_list)

['atheism',
 'atheists',
 'bible',
 'god',
 'keith',
 'moon',
 'religion',
 'sci',
 'sky',
 'space']